In [1]:
import openai
import ast
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df = pd.read_excel("path_to_data")
df.head()

,no,年齡,主要問題,DF1_a：早產,DF1_b：出生體重未達2500gm,DF2_a：染色體異常,DF2_b：頭顱顏面異常,DF2_f：先天性心臟病,DF3_b：母親妊娠期有不正常出血、糖尿病、妊娠毒血、梅毒、酗酒、抽煙,DF3_d：出生後新生兒有缺氧、無呼吸,自閉症,過動,亞斯伯格症候群,語言障礙,確診遲緩
0,1,2.97,內八、雙手無法向上伸直,0,0,0,0,0,0,0,0,0,0,0,1
1,2,3.04,過動、語言、認知,0,0,0,0,0,0,0,0,1,0,0,1
2,3,5.02,過動、專注力不集中,0,0,0,0,0,0,0,0,1,0,1,1
3,4,5.42,專注力不集中、語言表達不夠完整,0,0,0,0,0,0,0,0,0,0,0,1
4,5,4.61,說話不清楚,0,0,0,0,0,0,0,0,0,0,1,1


In [ ]:
api_key = 'your_api_key'

In [58]:
def both_data(age, problem, DF1_a, DF1_b, DF2_a, DF2_b, DF2_f, DF3_b, DF3_d):

    DF1_a = "Yes" if DF1_a == 1 else "No"
    DF1_b = "Yes" if DF1_b == 1 else "No"
    DF2_a = "Yes" if DF2_a == 1 else "No"
    DF2_b = "Yes" if DF2_b == 1 else "No"
    DF2_f = "Yes" if DF2_f == 1 else "No"
    DF3_b = "Yes" if DF3_b == 1 else "No"
    DF3_d = "Yes" if DF3_d == 1 else "No"

    result_list = list()
    try_time = 0

    bg = f"""
    Given the information provided for Child A:
    Age: {age}
    Premature birth: {DF1_a}
    Birth weight below 2500g: {DF1_b}
    Chromosomal abnormalities: {DF2_a}
    Craniofacial abnormalities: {DF2_b}
    Congenital heart disease: {DF2_f}
    Mother had abnormal bleeding, diabetes, toxemia of pregnancy, syphilis, alcohol abuse, smoking during pregnancy: {DF3_b}
    Newborn experienced hypoxia and apnea after birth: {DF3_d}
    """

    while(True):

        response = requests.post(
            'https://api.openai.com/v1/chat/completions',
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {api_key}'
            },
            json = {
                'model': 'gpt-4',
                'max_tokens': 500,
                'messages' : [
                    {"role": "system", "content": "Assistant is an autism expert to help users answer medical consulting problems according to children's background. Only answer questions in following python list format. Example output: [(probability)]"},
                    {"role": "user", "content": bg},
                    {"role": "assistant", "content": "I understand"},
                    {"role": "user", "content": f"""
                    Today, a doctor diagnosed Child A with {problem} based on the provided information.
                    """},
                    {"role": "assistant", "content": "I understand"},
                    {"role": "user", "content": f"""
                    developmental delay expert will provide the probabilities of Child A developing the developmental delay conditions: developmental delay.
                    Please only output the python list of probabilities.
                    """},
                ]
            })
        json = response.json()
        response_str = json['choices'][0]['message']['content']
    
        start = response_str.find('[')
        end = response_str.find(']', start + 1)
        try:
            if start == -1 or end == -1 or 0 in eval(response_str[int(start):int(end + 1)]) or 0.0 in eval(response_str[int(start):int(end + 1)]) or 0.00 in eval(response_str[int(start):int(end + 1)]) or None in eval(response_str[int(start):int(end + 1)]) or len(eval(response_str[int(start):int(end + 1)])) != 1:
                try_time += 1
                continue
            else:
                result_list = eval(response_str[int(start):int(end + 1)])
                break
        except:
            continue
    print(result_list)
    return result_list, try_time

In [ ]:
def background_only(age, problem, DF1_a, DF1_b, DF2_a, DF2_b, DF2_f, DF3_b, DF3_d):
    
    DF1_a = "Yes" if DF1_a == 1 else "No"
    DF1_b = "Yes" if DF1_b == 1 else "No"
    DF2_a = "Yes" if DF2_a == 1 else "No"
    DF2_b = "Yes" if DF2_b == 1 else "No"
    DF2_f = "Yes" if DF2_f == 1 else "No"
    DF3_b = "Yes" if DF3_b == 1 else "No"
    DF3_d = "Yes" if DF3_d == 1 else "No"
    
    result_list = list()
    try_time = 0
    
    bg = f"""
    Given the information provided for Child A:
    Age: {age}
    Premature birth: {DF1_a}
    Birth weight below 2500g: {DF1_b}
    Chromosomal abnormalities: {DF2_a}
    Craniofacial abnormalities: {DF2_b}
    Congenital heart disease: {DF2_f}
    Mother had abnormal bleeding, diabetes, toxemia of pregnancy, syphilis, alcohol abuse, smoking during pregnancy: {DF3_b}
    Newborn experienced hypoxia and apnea after birth: {DF3_d}
    """
    
    while(True):
        response = requests.post(
            'https://api.openai.com/v1/chat/completions',
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {api_key}'
            },
            json = {
                'model': 'gpt-3.5-turbo',
                'max_tokens': 500,
                'messages' : [
                    {"role": "system", "content": "Assistant is an autism expert to help users answer medical consulting problems according to children's background. Only answer questions in following python list format. Example output: [(probability)]"},
                    {"role": "user", "content": bg},
                    {"role": "assistant", "content": "I understand"},
                    {"role": "user", "content": f"""
                    Assuming you are a doctor, you will assess the probabilities of Child A developing the following condition: autism, according to the provided information above.
                    Please explain how do you come up with your results. And you MUST output the probabilities with python list at the end.
                    """},
                ]
            })
        json = response.json()
        response_str = json['choices'][0]['message']['content']
        start = response_str.find('[')
        end = response_str.find(']', start + 1)
        prob = response_str[int(start):int(end + 1)]
        try:
            if start == -1 or end == -1 or None in eval(prob) or len(eval(prob)) != 1:
                try_time += 1
                continue
            else:
                result_list = eval(prob)
                break
        except:
            continue
    return result_list, try_time

In [ ]:
def prescription_only(age, problem, DF1_a, DF1_b, DF2_a, DF2_b, DF2_f, DF3_b, DF3_d):
    
    DF1_a = "Yes" if DF1_a == 1 else "No"
    DF1_b = "Yes" if DF1_b == 1 else "No"
    DF2_a = "Yes" if DF2_a == 1 else "No"
    DF2_b = "Yes" if DF2_b == 1 else "No"
    DF2_f = "Yes" if DF2_f == 1 else "No"
    DF3_b = "Yes" if DF3_b == 1 else "No"
    DF3_d = "Yes" if DF3_d == 1 else "No"
    
    result_list = list()
    try_time = 0
    
    bg = f"""
    Given the information provided for Child A:
    Age: {age}
    Premature birth: {DF1_a}
    Birth weight below 2500g: {DF1_b}
    Chromosomal abnormalities: {DF2_a}
    Craniofacial abnormalities: {DF2_b}
    Congenital heart disease: {DF2_f}
    Mother had abnormal bleeding, diabetes, toxemia of pregnancy, syphilis, alcohol abuse, smoking during pregnancy: {DF3_b}
    Newborn experienced hypoxia and apnea after birth: {DF3_d}
    """
    
    while(True):
        
        response = requests.post(
            'https://api.openai.com/v1/chat/completions',
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {api_key}'
            },
            json = {
                'model': 'gpt-3.5-turbo',
                'max_tokens': 500,
                'messages' : [
                    {"role": "system", "content": f"""Assistant is an {condition} expert to help users answer medical consulting problems according to children's information, and will answer a probability ANYWAY. Only answer questions in following python list format. Example output: [(probability)]"""},
                    {"role": "user", "content": "now assistant a"},
                    {"role": "assistant", "content": "I understand"},
                    {"role": "user", "content": f"""
                    Today, a doctor diagnosed Child A with {problem} based on the provided information.
                    """},
                    {"role": "assistant", "content": "I understand"},
                    {"role": "user", "content": f"""
                    {condition} expert will provide the probabilities of Child A developing the {condition}.
                    Please only output the python list of probabilities.
                    """},
                ]
            })
        json = response.json()
        response_str = json['choices'][0]['message']['content']
        start = response_str.find('[')
        end = response_str.find(']', start + 1)
        prob = response_str[int(start):int(end + 1)]
        try:
            if start == -1 or end == -1 or None in eval(prob) or len(eval(prob)) != 1:
                try_time += 1
                continue
            else:
                result_list = eval(prob)
                break
        except:
            continue
    return result_list, try_time

In [ ]:
conditions = ['autism'] 
err_counter = 0
i=0
while i < df.shape[1]:
    child_prob = []
    for condition in conditions:
        n = 0
        while n < 1:
            age, problem, DF1_a, DF1_b, DF2_a, DF2_b, DF2_f, DF3_b, DF3_d = df["年齡"][i], df["主要問題"][i], df["DF1_a：早產"][i],df["DF1_b：出生體重未達2500gm"][i],df["DF2_a：染色體異常"][i],df["DF2_b：頭顱顏面異常"][i],df["DF2_f：先天性心臟病"][i],df["DF3_b：母親妊娠期有不正常出血、糖尿病、妊娠毒血、梅毒、酗酒、抽煙"][i],df["DF3_d：出生後新生兒有缺氧、無呼吸"][i]
            try:
                #function1:both_data, function2:background_only, function3:prescription_only
                lst, time = both_data(age, problem, DF1_a, DF1_b, DF2_a, DF2_b, DF2_f, DF3_b, DF3_d, condition)
                child_prob.append(lst[0])
                n+=1
            except Exception as e:
                pass
    final_lst.append(child_prob)
    i += 1 

In [63]:
#save the first row
df = pd.DataFrame(final_lst, columns=['1'])
df.to_excel('output.xlsx', index=False)